<a href="https://www.kaggle.com/code/marcjuniornkengue/resvidnet?scriptVersionId=91692201" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import Libraries and activate TPU
Importing the libraries and activate TPU to accelerate data processing and model training.

In [ ]:
#### Import libraries
import os
!pip install tensorflow_addons
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report
from scipy import stats
from sklearn.model_selection import train_test_split
import sklearn
import itertools
from tensorflow_addons.optimizers import CyclicalLearningRate
import matplotlib as mpl
mpl.style.use('seaborn')
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import logging
from random import choice
import random 
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# Set random seed
np.random.seed(123)

### Metrics functions plot
Plotting Accuracy and loss curve, for train and validation data.
The training loss indicates how well the model is fitting the training data, while the validation loss indicates how well the model fits validation data.
The training accuracy indicates how accurate the model is for the training data, while the validation accuracy indicates how accurate the model is for validation data.

In [ ]:
def metrics_plot(history, field, fn):
    def plot(data, val_data, best_index, best_value, title):
        plt.plot(range(1, len(data)+1), data, label='train')
        plt.plot(range(1, len(data)+1), val_data, label='validation')
        if not best_index is None:
            plt.axvline(x=best_index+1, linestyle=':', c="#777777")
        if not best_value is None:
            plt.axhline(y=best_value, linestyle=':', c="#777777")
        plt.xlabel('Epoch')
        plt.ylabel(field)
        plt.xticks(range(0, len(data), 20))
        plt.title(title)
        plt.legend()
        plt.savefig('{} curve.jpg'.format(field), format='jpg', dpi=300)
        plt.show()
        
        

    data = history.history[field]
    val_data = history.history['val_' + field]
    tail = int(0.15 * len(data))

    best_index = fn(val_data)
    best_value = val_data[best_index]

    plot(data, val_data, best_index, best_value, "{} over epochs (best {:06.4f})".format(field, best_value))
    #plot(data[-tail:], val_data[-tail:], None, best_value, "{} over last {} epochs".format(field, tail))

### Confusion matrix plot
The function print and plot the confusion matrix.
It's a specific table layout that allows visualization of the performance of the model. Each row of the matrix represents the instances in an actual class while each column represents the instances in a predicted class.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('{}.jpg'.format(title), format='jpg', dpi=300)

### ECG Signal classes
The class of the signals.
* Normal : Label for Normal ECG signal.
* HB : Label for abnormal ECG signal.
* PMI : Label for ECG signal of person with history of Myocardial Infarction.
* MI : Label for ECG signal of person with Myocardial Infarction.
* COVID : Label for ECG signal of person with COVID-19.

In [ ]:
classes=['Normal','HB','PMI', 'MI','COVID']

### Reading dataset and plot class repartition
Loading ECG dataset, and plotting data repartition per class.

In [ ]:
## Load data
df=pd.read_csv('../input/ecg-signal-covid/covid_dataset_balanced.csv',header=None)
df.reset_index(drop=True, inplace=True)

## Get features and label
x=df.iloc[:,:-1].values # Signal data
y=df.iloc[:,-1].values  # Signal label

print(df) # Display dataset

## Data repartition per class 
per_class = df.iloc[:,-1].value_counts() 
plt.figure(figsize=(20,10))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(per_class, labels=['Normal','HB','PMI', 'MI','COVID'], colors=['tab:blue','tab:orange','tab:purple','tab:olive','tab:green'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

### Train,test and validation split
The dataset is split into three subsets : 
* The train dataset, for training the model
* The validation dataset, to validate the model
* The test dataset, to evaluate the model

In [ ]:
y=y.astype(int) # Convert label into int
## Split into train and test
X_train, X_val, y_train, y_val = train_test_split(x,y, test_size=0.2, random_state=np.random.randint(1,1000, 1)[0], shuffle = True)
## Slip into test and validation
X_val, X_test, y_val, y_test  = train_test_split(X_val, y_val, test_size=0.5, random_state=np.random.randint(1,1000, 1)[0], shuffle = True)

## Expand data dimension for the Deep Learning Model
## Train data
X_train = np.expand_dims(X_train, -1)
y_train = np.expand_dims(y_train, -1)
## Validation data
X_val = np.expand_dims(X_val, -1)
y_val = np.expand_dims(y_val, -1)

## Test data
X_test = np.expand_dims(X_test, -1)
y_test = np.expand_dims(y_test, -1)


In [ ]:
data=x[0]
plt.plot(data)


### The ResvidNet Model
The model is inspired by ResNet18. Residual block of ResNet network is enhance by adding:
* A Batch Normalization Layer, to normalize all datas for training
* More Conv1D layers with higher filters numbers, to learning signal features better,faster and increase accuracy
* Dropout Layers and regularization Layers , in order to prevent overfitting.
The input of the overall model is a Conv1D layer, follow by a Gaussian Noise Layer was also added, to make the model more robust.

In [ ]:
def dnn_model(im_shape):
    inputs_cnn=keras.layers.Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=keras.layers.Dense(1024, activation='relu')(inputs_cnn)
    conv1_1=keras.layers.Dropout(0.3)(conv1_1)
    pool1=keras.layers.Dense(1024, activation='relu')(conv1_1)
    conv2_1=keras.layers.Dropout(0.3)(pool1)
    conv2_1=keras.layers.Dense(512, activation='relu')(conv2_1)
    pool2=keras.layers.Dropout(0.2)(conv2_1)
    conv3_1=keras.layers.Dense(128, activation='relu')(pool2)
    dense_end2 = keras.layers.Dense(64, activation='relu')(conv3_1)
    main_output = keras.layers.Dense(5, activation='softmax', name='main_output')(dense_end2)
    model = keras.Model(inputs= inputs_cnn, outputs=main_output)
    return model

In [ ]:
def cnn_model_ecg(im_shape):    
    inputs_cnn=keras.layers.Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=keras.layers.Convolution1D(64, (5), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1=keras.layers.BatchNormalization()(conv1_1)
    pool1=keras.layers.MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv1_1)
    conv2_1=keras.layers.Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool1)
    conv2_1=keras.layers.BatchNormalization()(conv2_1)
    pool2=keras.layers.MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=keras.layers.Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool2)
    conv3_1=keras.layers.BatchNormalization()(conv3_1)
    pool3=keras.layers.MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    flatten=keras.layers.Flatten()(pool3)
    flatten= keras.layers.Dropout(0.3)(flatten)
    dense_end1 = keras.layers.Dense(128, activation='relu')(flatten)
    dense_end2 = keras.layers.Dense(32, activation='relu')(dense_end1)
    main_output = keras.layers.Dense(5, activation='softmax', name='main_output')(dense_end2)
    model = keras.Model(inputs= inputs_cnn, outputs=main_output)
    return model

In [ ]:

def resnet(im_shape):
    ## The residual block of ResVidNet
    def residual_block(X, kernels, stride):
        out = keras.layers.Conv1D(kernels, stride,  kernel_regularizer=l2(0.001),padding='same')(X)
        out = keras.layers.ReLU()(out)
        
        out = keras.layers.Conv1D(kernels, stride,  kernel_regularizer=l2(0.001),padding='same')(out)
        out = keras.layers.ReLU()(out)
        out = keras.layers.Conv1D(kernels, stride, kernel_regularizer=l2(0.001) ,padding='same')(out)
        
        out = keras.layers.add([X, out])
        out = keras.layers.ReLU()(out)
        
        out = keras.layers.MaxPool1D(5, 2)(out)
        
        return out
    
    kernels = 64
    stride = 5


    inputs = keras.layers.Input(im_shape)
    X = keras.layers.Conv1D(kernels, stride)(inputs)
    #X = keras.layers.GaussianNoise(0.01)(X)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = keras.layers.Flatten()(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    output = keras.layers.Dense(5, activation='softmax')(X)

    model = keras.Model(inputs=inputs, outputs=output)
    return model

In [ ]:

def ResVidNet(im_shape):
    ## The residual block of ResVidNet
    def residual_block(X, kernels, stride):
        out = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(X)
        
        out = keras.layers.Conv1D(kernels, stride,  kernel_regularizer=l2(0.005),padding='same')(out)
        out = keras.layers.ReLU()(out)
        out = keras.layers.Dropout(0.1)(out) 
        
        out = keras.layers.Conv1D(kernels, stride,  kernel_regularizer=l2(0.005),padding='same')(out)
        out = keras.layers.ReLU()(out)
        out = keras.layers.Dropout(0.1)(out) 
        
        out = keras.layers.Conv1D(kernels, stride,  kernel_regularizer=l2(0.005),padding='same')(out)
        out = keras.layers.ReLU()(out)
        out = keras.layers.Dropout(0.1)(out) 
        
        out = keras.layers.Conv1D(kernels, stride,  kernel_regularizer=l2(0.005),padding='same')(out)
        out = keras.layers.Dropout(0.1)(out) 
        out = keras.layers.ReLU()(out)
        
        out = keras.layers.Conv1D(kernels, stride, kernel_regularizer=l2(0.005) ,padding='same')(out)
        
        out = keras.layers.add([X, out])
        out = keras.layers.ReLU()(out)
        out = keras.layers.MaxPool1D(5, 2)(out)
        
        return out
    
    kernels = 256
    stride = 15


    inputs = keras.layers.Input(im_shape,name='main_output')
    X = keras.layers.Conv1D(kernels, stride)(inputs)
    X = keras.layers.GaussianNoise(0.01)(X)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = residual_block(X, kernels, stride)
    X = keras.layers.Flatten()(X)
    X = keras.layers.Dense(128, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    output = keras.layers.Dense(5, activation='softmax',name="last_conv")(X)

    model = keras.Model(inputs=inputs, outputs=output)
    return model

### Setting the hyper-parameters, compiling and training the model

In [ ]:
y_train.shape

In [ ]:
### Hyper parameters
N=X_train.shape[0]
batch_size = 128 * tpu_strategy.num_replicas_in_sync
iterations = N/batch_size
step_size= 2 * iterations
lr_schedule = CyclicalLearningRate(1e-5, 1e-3, step_size=step_size, scale_fn=lambda x: tf.pow(0.95,x))
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
with tpu_strategy.scope():
        convidnet_model = ResVidNet((X_train.shape[1],1)) # Instanciate ResVidNet
        resnet_model = resnet((X_train.shape[1],1)) # Instanciate ResVidNet
        cnn_model = cnn_model_ecg((X_train.shape[1],1)) # Instanciate ResVidNet
        d_model = dnn_model((X_train.shape[1],1)) # Instanciate ResVidNet
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=80, verbose=1) # Early stopping parameters
        save_best_weights_resvidnet = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True) # Model checkpoint parameters
        save_best_weights_resnet = ModelCheckpoint(filepath="weights_resnet.hdf5", verbose=1, save_best_only=True) # Model checkpoint parameters
        save_best_weights_cnn = ModelCheckpoint(filepath="weights_cnn.hdf5", verbose=1, save_best_only=True) # Model checkpoint parameters
        save_best_weights_dnn = ModelCheckpoint(filepath="weights_dnn.hdf5", verbose=1, save_best_only=True) # Model checkpoint parameters
        
        convidnet_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Model compiling
        resnet_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Model compiling
        cnn_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Model compiling
        d_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Model compiling
        
        
        #convidnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy']) # Model compiling

## Model training
history_resvidnet = convidnet_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=False,callbacks=[save_best_weights_resvidnet,early_stopping])
history_resnet = resnet_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=False,callbacks=[save_best_weights_resnet,early_stopping])
history_cnn = cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=False,callbacks=[save_best_weights_cnn,early_stopping])
#history_dnn = d_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, shuffle=False,callbacks=[save_best_weights_dnn,early_stopping])



### Model evaluation
The model is evaluate using test data

In [ ]:
convidnet_model.load_weights('weights.hdf5') # Loading model
X_covid_test = convidnet_model.evaluate(X_test, y_test) # Evaluate model

resnet_model.load_weights('weights_resnet.hdf5') # Loading model
X_resnet_test = resnet_model.evaluate(X_test, y_test) # Evaluate model

cnn_model.load_weights('weights_cnn.hdf5') # Loading model
X_cnn_test = cnn_model.evaluate(X_test, y_test) # Evaluate model
"""
dnn_model.load_weights('weights_dnn.hdf5') # Loading model
X_dnn_test = dnn_model.evaluate(X_test, y_test) # Evaluate model"""

### Plotting Loss and Accuracy

In [ ]:
metrics_plot(history_resvidnet, 'loss', lambda x: np.argmin(x))
metrics_plot(history_resvidnet, 'accuracy', lambda x: np.argmax(x))

### Plotting confusion matrix

In [ ]:
y_pred = tf.argmax(convidnet_model.predict(X_test), axis=-1)
# Compute confusion matrix
cnf_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Normal','HB','PMI', 'MI','COVID'],title=' Confusion matrix')

y_pred1 = tf.argmax(resnet_model.predict(X_test), axis=-1)
# Compute confusion matrix
cnf_matrix1 = sklearn.metrics.confusion_matrix(y_test, y_pred1)
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix1, classes=['Normal','HB','PMI', 'MI','COVID'],title=' Confusion matrix ResNet')

y_pred2 = tf.argmax(cnn_model.predict(X_test), axis=-1)
# Compute confusion matrix
cnf_matrix2 = sklearn.metrics.confusion_matrix(y_test, y_pred2)
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix2, classes=['Normal','HB','PMI', 'MI','COVID'],title=' Confusion matrix CNN')
"""
y_pred3 = tf.argmax(dnn_model.predict(X_test), axis=-1)
# Compute confusion matrix
cnf_matrix3 = sklearn.metrics.confusion_matrix(y_test, y_pred3)
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix3, classes=['Normal','HB','PMI', 'MI','COVID'],title=' Confusion matrix')"""

### Classification report
It's a performance evaluation metric of the quality of predictions from ResVidNet. It's shows the precision, recall, F1 Score, and support.

In [ ]:
df = pd.DataFrame(classification_report(y_pred, y_test, digits=2,output_dict=True)).T
df['support'] = df.support.apply(int)
df.rename(index={'0':'Normal', '1':'Abnormal HB', '2':'History of MI' ,'3':'Myocardial Infarction (MI)','4': 'COVID-19'}, inplace=True)
df = df[0:5]
df.style.background_gradient(cmap='viridis')


In [ ]:

df1 = pd.DataFrame(classification_report(y_pred1, y_test, digits=2,output_dict=True)).T
df1['support'] = df1.support.apply(int)
df1.rename(index={'0':'Normal', '1':'Abnormal HB', '2':'History of MI' ,'3':'Myocardial Infarction (MI)','4': 'COVID-19'}, inplace=True)
df1 = df1[0:5]
df1.style.background_gradient(cmap='viridis')

In [ ]:
df2 = pd.DataFrame(classification_report(y_pred2, y_test, digits=2,output_dict=True)).T
df2['support'] = df2.support.apply(int)
df2.rename(index={'0':'Normal', '1':'Abnormal HB', '2':'History of MI' ,'3':'Myocardial Infarction (MI)','4': 'COVID-19'}, inplace=True)
df2 = df2[0:5]
df2.style.background_gradient(cmap='viridis')

In [ ]:
"""
df = pd.DataFrame(classification_report(y_pred3, y_test, digits=2,output_dict=True)).T
df['support'] = df.support.apply(int)
df.rename(index={'0':'Normal', '1':'Abnormal HB', '2':'History of MI' ,'3':'Myocardial Infarction (MI)','4': 'COVID-19'}, inplace=True)
df = df[0:5]
df.style.background_gradient(cmap='viridis')"""

In [ ]:
def grad_cam(layer_name, data):
    grad_model = tf.keras.models.Model(
        [convidnet_model.inputs], [convidnet_model.get_layer(layer_name).output, convidnet_model.output]
    )
    last_conv_layer_output, preds = grad_model(data)
    
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(data)
        pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
        
    grads = tape.gradient(class_channel, last_conv_layer_output)
    
    pooled_grads = tf.reduce_mean(grads,axis=(0))
    
    last_conv_layer_output = last_conv_layer_output[0]
    
    heatmap = last_conv_layer_output * pooled_grads
    #heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    heatmap = np.expand_dims(heatmap,0)
    return heatmap

In [ ]:
layer_name = "last_conv"
label = ['Normal','HB','PMI', 'MI','COVID']
cnt = 0
for i in X_val[:100]:
        pred = convidnet_model.predict(np.expand_dims(i,axis=0))
        index=np.argmax(pred[0])
        heatmap = grad_cam(layer_name,np.expand_dims(i,axis=0))
        print(f"Model prediction = {label[index]} ({pred[0][index]*100}), True label = {label[int(y_val[cnt])]}")
        plt.figure(figsize=(30,4))
        plt.imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,1250,i.min(),i.max()], alpha=0.5)
        plt.plot(i,'k')
        plt.colorbar()
        plt.show()
        cnt +=1

In [ ]:
layer_name = "last_conv"
label = ['Normal','HB','PMI', 'MI','COVID']
cnt = 0
T=[X_val[3],X_val[5],X_val[6],X_val[8],X_val[23]]
"""
for i,j in zip(T,[4,7,14,38,49]):    
        pred = convidnet_model.predict(np.expand_dims(i,axis=0))
        index=np.argmax(pred[0])
        heatmap = grad_cam(layer_name,np.expand_dims(i,axis=0))
        plt.figure(figsize=(30,4))
        plt.imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,375,i.min(),i.max()], alpha=0.5)
        plt.title(f"Model prediction = {label[index]} ({pred[0][index]*100})%,True label = {label[int(y_val[j])]}")
        plt.plot(i,'k')
        plt.colorbar()
        plt.show()
        #plt.savefig('{}.eps'.format(cnt), format='eps', dpi=300)
        cnt +=1
"""
fig, axs = plt.subplots(5, 1,figsize=(15,15))
pred = convidnet_model.predict(np.expand_dims(T[0],axis=0))
index=np.argmax(pred[0])
heatmap = grad_cam(layer_name,np.expand_dims(T[0],axis=0))
pcm=axs[0].imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,1250,T[0].min(),T[0].max()], alpha=0.5)
axs[0].set_title(f"Model prediction = {label[index]} ({pred[0][index]*100})%, True label ={label[int(y_val[3])]}")
axs[0].plot(T[0],'k')
fig.colorbar(pcm, ax=axs[0])

pred = convidnet_model.predict(np.expand_dims(T[1],axis=0))
index=np.argmax(pred[0])
heatmap = grad_cam(layer_name,np.expand_dims(T[1],axis=0))
pcm=axs[1].imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,1250,T[1].min(),T[1].max()], alpha=0.5)
axs[1].set_title(f"Model prediction = {label[index]} ({pred[0][index]*100})%, True label = {label[int(y_val[5])]}")
axs[1].plot(T[1],'k')
fig.colorbar(pcm, ax=axs[1])

pred = convidnet_model.predict(np.expand_dims(T[2],axis=0))
index=np.argmax(pred[0])
heatmap = grad_cam(layer_name,np.expand_dims(T[2],axis=0)) 
pcm=axs[2].imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,1250,T[2].min(),T[2].max()], alpha=0.5)
axs[2].set_title(f"Model prediction = {label[index]} ({pred[0][index]*100})%, True label = {label[int(y_val[6])]}")
axs[2].plot(T[2],'k')
fig.colorbar(pcm, ax=axs[2])

pred = convidnet_model.predict(np.expand_dims(T[3],axis=0))
index=np.argmax(pred[0])
heatmap = grad_cam(layer_name,np.expand_dims(T[3],axis=0))
pcm=axs[3].imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,1250,T[3].min(),T[3].max()], alpha=0.5)
axs[3].set_title(f"Model prediction = {label[index]} ({pred[0][index]*100})%, True label = {label[int(y_val[8])]}")
axs[3].plot(T[3],'k')
fig.colorbar(pcm, ax=axs[3])

pred = convidnet_model.predict(np.expand_dims(T[4],axis=0))
index=np.argmax(pred[0])
heatmap = grad_cam(layer_name,np.expand_dims(T[4],axis=0))
pcm=axs[4].imshow(np.expand_dims(heatmap,axis=2),cmap='Reds', aspect="auto", interpolation='nearest',extent=[0,1250,T[4].min(),T[4].max()], alpha=0.5)
axs[4].set_title(f"Model prediction = {label[index]} ({pred[0][index]*100})%, True label = {label[int(y_val[23])]}")
axs[4].plot(T[4],'k')
fig.colorbar(pcm, ax=axs[4])

fig.tight_layout()
plt.savefig('GRADCAM.jpg',format='jpg',dpi=300)
plt.show()

In [ ]:
val_data = history_resvidnet.history['val_accuracy']
val_data1 = history_resnet.history['val_accuracy']
val_data2 = history_cnn.history['val_accuracy']

plt.plot(range(1, len(val_data)+1), val_data, label='ResvidNet')
plt.plot(range(1, len(val_data1)+1), val_data1, label='ResNet')
plt.plot(range(1, len(val_data2)+1), val_data2, label='CNN')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(range(0, 100, 10))
plt.title('Accuracy curves')
plt.legend()
plt.savefig('Accuracy curves.jpg', format='jpg', dpi=300)
plt.show()
    
val_data = history_resvidnet.history['val_loss']
val_data1 = history_resnet.history['val_loss']
val_data2 = history_cnn.history['val_loss']

plt.plot(range(1, len(val_data)+1), val_data, label='ResvidNet')
plt.plot(range(1, len(val_data1)+1), val_data1, label='ResNet')
plt.plot(range(1, len(val_data2)+1), val_data2, label='CNN')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(range(0, 100, 10))
plt.title('Loss curves')
plt.legend()
plt.savefig('Loss curves.jpg', format='jpg', dpi=300)
plt.show()
    


In [ ]:
from PIL import Image
image = Image.open('./GRADCAM.jpg')
im= image.convert('CMYK')
im.save('GRADCAM.jpeg')

image = Image.open('./loss curve.jpg')
im= image.convert('CMYK')
im.save('loss curve.jpeg')

image = Image.open('./accuracy curve.jpg')
im= image.convert('CMYK')
im.save('accuracy curve.jpeg')

image = Image.open('./Loss curves.jpg')
im= image.convert('CMYK')
im.save('loss curves.jpeg')

image = Image.open('./Accuracy curves.jpg')
im= image.convert('CMYK')
im.save('accuracy curves.jpeg')

image = Image.open('./ Confusion matrix.jpg')
im= image.convert('CMYK')
im.save('Confusion matrix.jpeg')

image = Image.open('./ Confusion matrix ResNet.jpg')
im= image.convert('CMYK')
im.save('Confusion matrix ResNet.jpeg')

image = Image.open('./ Confusion matrix CNN.jpg')
im= image.convert('CMYK')
im.save('Confusion matrix CNN.jpeg')